This notebook includes the main procedures of cross-match between ICRF3 and Gaia-EDR3 catalogs.

In [1]:
import time
import numpy as np
from astropy.table import Table, join, Column
from astropy import units as u

# My progs
from my_progs.catalog.read_icrf import read_icrf3
from my_progs.catalog.read_gaia import read_edr3_icrf_sou, read_dr2_iers
from my_progs.catalog.pos_diff import radio_cat_diff_calc

Get the Gaia EDR3 data.

In [2]:
gedr3 = read_edr3_icrf_sou(True)

Crossmatch with ICRF3 S/X catalog.

In [3]:
icrf3sx = read_icrf3(wv="sx")

Calculate positional offset between ICRF3 S/X catalog and Gaia EDR3, in sense of S/X minus EDR3.

In [4]:
pos_oft = radio_cat_diff_calc(icrf3sx, gedr3, sou_name="icrf_name")
pos_oft.remove_columns(['ra_err_1', 'dec_err_1', 'ra', 'ra_err_2', 'dec','dec_err_2'])

pos_oft1 = radio_cat_diff_calc(gedr3, icrf3sx, sou_name="icrf_name")
pos_oft1.remove_columns(['ra_err_1', 'dec_err_1', 'ra', 'ra_err_2', 'dec','dec_err_2'])

Join the positional offset table with original catalogs for further access.

In [5]:
gedr3_icrf3sx = join(gedr3, icrf3sx, keys='icrf_name', table_names=['gedr3', 'icrf3sx'])
gedr3_icrf3sx_oft = join(gedr3_icrf3sx, pos_oft, keys='icrf_name')

gedr3_icrf3sx_oft.rename_columns(['ra_gedr3', 'dec_gedr3'],
                                 ['ra', 'dec'])

In [6]:
icrf3sx_gedr3_oft = join(gedr3_icrf3sx, pos_oft1, keys='icrf_name')
icrf3sx_gedr3_oft.rename_columns(['ra_icrf3sx', 'dec_icrf3sx'],
                                 ['ra', 'dec'])

Save the data.

In [7]:
gedr3_icrf3sx_oft.write('../data/icrf3sx_gedr3.fits', overwrite=True)
icrf3sx_gedr3_oft.write('../data/gedr3_icrf3sx.fits', overwrite=True)

Repeat the same procedures for K and X/Ka catalogs.

In [8]:
icrf3k = read_icrf3(wv="k")

pos_oft = radio_cat_diff_calc(icrf3k, gedr3, sou_name="icrf_name")
pos_oft.remove_columns(['ra_err_1', 'dec_err_1', 'ra',
                        'ra_err_2', 'dec', 'dec_err_2'])

gedr3_icrf3k = join(gedr3, icrf3k, keys='icrf_name',
                    table_names=['gedr3', 'icrf3k'])
gedr3_icrf3k_oft = join(gedr3_icrf3k, pos_oft, keys='icrf_name')
gedr3_icrf3k_oft.rename_columns(['ra_gedr3', 'dec_gedr3'],
                                ['ra', 'dec'])

gedr3_icrf3k_oft.write('../data/icrf3k_gedr3.fits', overwrite=True)

In [9]:
icrf3xka = read_icrf3(wv="xka")

pos_oft = radio_cat_diff_calc(icrf3xka, gedr3, sou_name="icrf_name")
pos_oft.remove_columns(['ra_err_1', 'dec_err_1', 'ra',
                        'ra_err_2', 'dec', 'dec_err_2'])

gedr3_icrf3xka = join(gedr3, icrf3xka, keys='icrf_name',
                      table_names=['gedr3', 'icrf3xka'])
gedr3_icrf3xka_oft = join(gedr3_icrf3xka, pos_oft, keys='icrf_name')

gedr3_icrf3xka_oft.rename_columns(['ra_gedr3', 'dec_gedr3'],
                                  ['ra', 'dec'])

gedr3_icrf3xka_oft.write('../data/icrf3xka_gedr3.fits', overwrite=True)

Crossmatch with Gaia DR2 IERS sample.

In [10]:
gdr2 = read_dr2_iers()

In [11]:
pos_oft = radio_cat_diff_calc(gdr2, gedr3, sou_name='iers_name')
pos_oft.remove_columns(['ra_err_1', 'dec_err_1', 'ra',
                        'ra_err_2', 'dec', 'dec_err_2'])

gedr3_dr2 = join(gedr3, gdr2, keys='iers_name',
                 table_names=['gedr3', 'gdr2'])
gedr3_dr2_oft = join(gedr3_dr2, pos_oft)

gedr3_dr2_oft.rename_columns(['ra_gedr3', 'dec_gedr3'],
                             ['ra', 'dec'])

gedr3_dr2_oft.write('../data/gdr2_gedr3.fits', overwrite=True)

Collect the multi-frequency position offsets for the common sample.

In [12]:
gedr3_icrf3sx_oft.keep_columns([
    'source_id',
    'ra', 'ra_err_gedr3', 'dec', 'dec_err_gedr3',
    'parallax', 'parallax_err', 'parallax_over_error',
    'pm', 'pmra', 'pmra_err', 'pmdec', 'pmdec_err',
    'ra_dec_corr_gedr3', 'astrometric_n_obs_al',
    'astrometric_gof_al', 'astrometric_chi2_al',
    'astrometric_matched_transits', 'visibility_periods_used',
    'astrometric_sigma5d_max', 'ruwe',
    'duplicated_source', 'phot_g_mean_mag',
    'phot_bp_mean_mag', 'phot_rp_mean_mag',
    'bp_rp', 'bp_g', 'g_rp', 'l', 'b',
    'ecl_lon', 'ecl_lat',
    'considered_for_reference_frame_orientation',
    'used_for_reference_frame_orientation',
    'icrf_name', 'iers_name_gedr3',
    'pos_err_max_gedr3', 'pos_err_min_gedr3', 'eepa_gedr3',
    'type',
    'ra_icrf3sx', 'dec_icrf3sx', 'ra_err_icrf3sx', 'dec_err_icrf3sx',
    'ra_dec_corr_icrf3sx', 'mean_obs',
    'pos_err_max_icrf3sx', 'pos_err_min_icrf3sx', 'eepa_icrf3sx',
    'beg_obs', 'end_obs', 'nb_sess', 'nb_del',
    'dra', 'ddec', 'dra_err', 'ddec_err', 'dra_ddec_cov',
    'ang_sep', 'ang_sep_err', 'pa', 'pa_err',
    'nor_ra', 'nor_dec', 'nor_sep'])

gedr3_icrf3sx_oft.rename_column('iers_name_gedr3', 'iers_name')

In [13]:
gedr3_icrf3k_oft.keep_columns([
    'source_id', 'ra_icrf3k', 'dec_icrf3k',
    'ra_err_icrf3k', 'dec_err_icrf3k', 'ra_dec_corr_icrf3k',
    'mean_obs', 'pos_err_max_icrf3k',
    'pos_err_min_icrf3k', 'eepa_icrf3k',
    'beg_obs', 'end_obs', 'nb_sess', 'nb_del',
    'dra', 'ddec', 'dra_err', 'ddec_err', 'dra_ddec_cov',
    'ang_sep', 'ang_sep_err', 'pa', 'pa_err',
    'nor_ra', 'nor_dec', 'nor_sep'])

In [14]:
gedr3_icrf3xka_oft.keep_columns([
    'source_id', 'ra_icrf3xka', 'dec_icrf3xka',
    'ra_icrf3xka', 'dec_icrf3xka',
    'ra_err_icrf3xka', 'dec_err_icrf3xka', 'ra_dec_corr_icrf3xka',
    'mean_obs', 'pos_err_max_icrf3xka', 
    'pos_err_min_icrf3xka', 'eepa_icrf3xka',
    'beg_obs', 'end_obs', 'nb_sess', 'nb_del',
    'dra', 'ddec', 'dra_err', 'ddec_err', 'dra_ddec_cov',
    'ang_sep', 'ang_sep_err', 'pa', 'pa_err',
    'nor_ra', 'nor_dec', 'nor_sep'])

In [15]:
gedr3_dr2_oft.keep_columns([
    'source_id_gedr3', 'ra_gdr2', 'dec_gdr2',
    'ra_err_gdr2', 'dec_err_gdr2', 'ra_dec_corr_gdr2',
    'pos_err_max_gdr2', 'pos_err_min_gdr2', 'eepa_gdr2',
    'dra', 'ddec', 'dra_err', 'ddec_err', 'dra_ddec_cov',
    'ang_sep', 'ang_sep_err', 'pa', 'pa_err',
    'nor_ra', 'nor_dec', 'nor_sep'])

gedr3_dr2_oft.rename_column('source_id_gedr3', 'source_id')

In [16]:
temp1 = join(gedr3_icrf3sx_oft, gedr3_icrf3k_oft, 
             keys="source_id", table_names=["sx", "k"])

temp2 = join(gedr3_icrf3xka_oft, gedr3_dr2_oft, 
             keys="source_id", table_names=["xka", "dr2"],
            join_type="left")

multi_freq_oft = join(temp1, temp2, keys="source_id")

In [17]:
multi_freq_oft.meta["comments"] = [
    "Position offset for {} common sources between ICRF3 (all 3 bands) and Gaia-CRF2"
    "wrt. Gaia EDR3".format(len(multi_freq_oft)),
    "Created at {}".format(time.asctime())]

multi_freq_oft.write("../data/multi_pos_oft_wrt_edr3.fits", overwrite=True)

In [18]:
multi_freq_oft.meta["comments"] = [
    "List of {} common sources between ICRF3 (all 3 bands) and Gaia-CRF3".format(len(multi_freq_oft)),
    "Created at {}".format(time.asctime())]
multi_freq_oft.write("../data/com-sou-list.txt", format="ascii",
                    include_names=["iers_name"], overwrite=True)

A crossmatch betweem ICRF3 K, ICRF3 X/Ka and ICRF3 S/X is also made.

In [19]:
# pos_oft = radio_cat_diff_calc(icrf3k, icrf3sx, sou_name="iers_name")
# pos_oft.remove_columns(['ra_err_1', 'dec_err_1', 'ra',
#                         'ra_err_2', 'dec', 'dec_err_2'])

# icrf3sx_icrf3k = join(icrf3sx, icrf3k, keys='iers_name',
#                       table_names=['icrf3sx', 'icrf3k'])
# icrf3sx_icrf3k_oft = join(icrf3sx_icrf3k, pos_oft, keys='iers_name')
# icrf3sx_icrf3k_oft.rename_columns(['ra_icrf3sx', 'dec_icrf3sx'],
#                                   ['ra', 'dec'])

# icrf3sx_icrf3k_oft.write('../data/icrf3k_icrf3sx.fits', overwrite=True)

In [20]:
# pos_oft = radio_cat_diff_calc(icrf3xka, icrf3sx, sou_name="iers_name")
# pos_oft.remove_columns(['ra_err_1', 'dec_err_1', 'ra',
#                         'ra_err_2', 'dec', 'dec_err_2'])

# icrf3sx_icrf3xka = join(icrf3sx, icrf3xka, keys='iers_name',
#                       table_names=['icrf3sx', 'icrf3xka'])
# icrf3sx_icrf3xka_oft = join(icrf3sx_icrf3xka, pos_oft, keys='iers_name')
# icrf3sx_icrf3xka_oft.rename_columns(['ra_icrf3sx', 'dec_icrf3sx'],
#                                   ['ra', 'dec'])

# icrf3sx_icrf3xka_oft.write('../data/icrf3xka_icrf3sx.fits', overwrite=True)